- IMGRGB_0112.xlsx와 TFres_0112re.csv을 기반으로 수행한다  // Slice_img05.ipynb
더하여 처리 안된 이미지는 수동으로 작업함
seperIMG_notM02.ipynb
- 01/16
/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/Split_ori 

#import library
- colab에서 이용하기 위해 사용된 코드 존재(이 부분은 제거하여야 함) --> 제거한 파일도 따로 제작하여 드릴 예정

In [9]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)

Mounted at /content/gdrive/


In [10]:
!pip install callee
#for excuse error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import numpy as np
import tensorflow as tf
import pandas as pd

from PIL import Image
import os
from glob import glob
import shutil
from callee import *
import math

import shutil
import random

#dir list
- 디렉터리에 따른 자료
  - WSI 원본 ) 
    - in server 
    It is located in "Pangyo server"
    original images:  /HDD2/HnE_images/original/CNU/LUAD/MRXS/
    mask images: HDD2/HnE_images/processing/CNU/LUAD/mask/usable/tiff/
    Annotation images: HDD2/HnE_images/processing/CNU/LUAD/mask/tumor/tiff/
    - 개인 구글 드라이브 위치 
    /TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/tumor_prev

  - slide IMG )
    - /TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/Split_ori/ALL/

In [48]:
#about oriimg
base_dir="/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/Split_ori/" #unmask+mask img slies
ORIImg="/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_orig/tumor_prev/" #original WSI이미지
prev_slice=base_dir+"ALL/"#WSI 이미지 slice 저장
oribase_dir="/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/DeepL/"

#about 2img
ori_dir="/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/"

#about learningimg
test_dir="/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/test/"
train_dir="/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/train/"
validation_dir="/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/validation/"

#IMG slice


##Slice img 
- 저의 경우 갯수를 지정하여 unmasked/ masked이미지를 동시에 잘랐습니다.(같은 부분의 numbering이 똑같이 되기 때문) 이렇게 될 경우 각 원본 이미지에 따른 crop된 이미지 크기가 제각각이 되게 되는 단점이 있습니다.

In [18]:
#original WSI이미지를 자르기 위한 코드
w_num=30 ; h_len=30#image slice # (갯수로 지정함)

def image_crop(infilename ,save_path): #(infilename ,save_path)
    img = Image.open(ORIImg+infilename )
    (img_h, img_w) = img.size #width, height  ;#print(img.size)

    wslice_size=(img.width)/w_num
    hslice_size=(img.height)/h_len

    grid_w = wslice_size # crop width
    grid_h = hslice_size # crop height
    range_w = (int)(img_w/grid_w)
    range_h = (int)(img_h/grid_h)
 
    i = 0
    for w in range(range_w):
        for h in range(range_h):
            bbox = (h*grid_h, w*grid_w, (h+1)*(grid_h), (w+1)*(grid_w))   # 가로 세로 시작, 가로 세로 끝
            crop_img = img.crop(bbox)
 
            fname = "{}".format("{0:04d}".format(i))
            savename = save_path + fname +"_"+infilename
            
           ## crop_img.save(savename) # 활성시 원하는 크기로 지정한 이름명과 디렉터리에 저장되게 된다.
            print('save file ' + savename)
            i += 1

In [35]:
for (path,dir,file) in os.walk(ORIImg):
  #if(path.rfind("masked")!=-1):
  if(len(dir)==0):
    infilename=path[int(path.rfind("/"))+1:] ; 
    print("파일 이름",infilename)#original img 
    ##image_crop(infilename, ori_dir)

파일 이름 87-1+masked.jpg
파일 이름 87-1+original.jpg
파일 이름 2-1+masked.jpg
파일 이름 127-1+masked.jpg
파일 이름 141-1+original.jpg
파일 이름 2-1+original.jpg
파일 이름 74-2+original.jpg
파일 이름 127-1+original.jpg
파일 이름 74-2+masked.jpg
파일 이름 141-1+masked.jpg
파일 이름 94-3+original.jpg
파일 이름 94-3+masked.jpg


#Crop한 이미지 전처리
- crop한 결과 슬라이드를 보시면 샘플 부분이 아예 없거나 의미없을 정도로 적은 경우도 많을 것입니다. 이런 경우 유의미한 데이터에 비하여 데이터 크기가 작은 경우가 많습니다.


저는 이런 점을 이용하여 리눅스 기능을 이용하였습니다.(아래 코드 작동 확인시 의도치 않게 제거 될 수 있습니다. 제 드라이브를 이용신 경우 복구 해드릴 수 있지만 타 환경 이용 중 실수 하신건 못 도아드립니다.)


그리고 제거 전에 미리 cancer와 normal를 먼저 구분해 두시는 것이 데이터 안정성에 좋습니다.(이후에 무의미 데이터 제거 해 주세요)

- 가능하시면 Split_ori/ALL은 건들지 말아주세요.

In [ ]:
#%cd  /content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/Split_ori/UnMCancer/
#%ls -lSh
#%pwd
#%rm -r *

##!find ./* -size -1800c -size +1c 
#!find ./* -size -1899c -size +1c  -exec rm -rf {} \;

#Data Setup
- 디렉터리의 파일의 갯수 확인과 의미 있는 데이터만 남기기 위함 (gdrive에서 작동시 이미 데이터가 있으므로 작동안됨)

In [36]:
print(os.listdir(base_dir))

def img_mv(b_dir,dir_name,list_name):
  if len((os.listdir(dir_name)))==0 :#파일이 빈 경우 만 이용
    #print('b_dir :',b_dir,"dir_name :",dir_name)
    for fn in list_name:
      try:
        print("fn :",fn)
        ##shutil.copy(b_dir+fn,dir_name+fn) #copy file
      except:
        continue
    del tmplist[0]
  else:
    del tmplist[0]
    print("already made",len(os.listdir(dir_name)))

['ALL', 'Masked', 'TMasked', 'UnMasked', 'TUnMasked']


In [42]:
re_ftotal=[""]#total에서 masked되고 crop된 이미지의 cancer img list저장을 위한 것

#/content/gdrive/MyDrive/MPbio_GD/ProG/TIPS/OWN/adenocarcinoma_DeepL/IMG/Mask_01/slice_img/F_res/Split_ori/Masked/MCancer/
for (path,dir,file) in os.walk(base_dir+"Masked/MCancer/"): 
  #if (len(dir)!=False):# and (path.rfind("masked")!=-1):
  for i in file:
    re_ftotal.append(i)
  del re_ftotal[0]

In [51]:
for (path,dir,file) in os.walk(base_dir+"UnMasked/"):
  if (len(dir)==0):# and (path.rfind("masked")!=-1):
    infilename=path[int(path.rfind("/"))+1:]
    print("infilename",infilename)
    image_crop(infilename,ori_dir) 

In [ ]:
  #slice이미지(Masked)를 Masked Cancer폴더로 복사
#img_mv(base_dir+"TMasked/",base_dir+"Masked/MCancer/",re_ftotal)

##masked img에서 분류해둔 info를 통해 unmaksed img와 매칭 시킨다
- masked IMG의 경우 전문가 표시하였기에 일반인도 구분가능하므로 masked이미지의 info를 뽑고 unmasked에 같은 이름을 붙임

In [62]:
def img_mv2(b_dir,dir_name,st_list,list_name):
  #clear #insert
  print("list_name:",list_name,"\nst_list:",st_list)
  for fn in list_name:
    try:
      #if len((os.listdir(dir_name)))==0 :
      print('dir_name :',dir_name+fn)
      #shutil.copy(b_dir+fn,dir_name+fn)#copy file
    except:
      continue

In [65]:
tum_list=[""];num_list=[""];tt_list=[""]
tm_list=os.listdir(base_dir+"Masked/MCancer/")
nm_list=os.listdir(base_dir+"Masked/MNormal/")

for i in nm_list:
  num_list.append(i.split("+")[0]+"+original.jpg")
  tt_list.append(i.split("+")[0])
del num_list[0]
del tt_list[0]
img_mv2(ori_dir+"UnMasked/",ori_dir+"UnMNormal/",tt_list,num_list)

list_name: ['0382_2-1+original.jpg', '0383_127-1+original.jpg', '0383_141-1+original.jpg', '0383_2-1+original.jpg', '0383_74-2+original.jpg', '0383_94-3+original.jpg', '0384_2-1+original.jpg', '0384_94-3+original.jpg', '0385_127-1+original.jpg', '0385_2-1+original.jpg', '0385_74-2+original.jpg', '0385_94-3+original.jpg', '0386_127-1+original.jpg', '0386_2-1+original.jpg', '0386_74-2+original.jpg', '0386_94-3+original.jpg', '0387_74-2+original.jpg', '0387_94-3+original.jpg', '0388_74-2+original.jpg', '0389_74-2+original.jpg', '0389_94-3+original.jpg', '0390_74-2+original.jpg', '0390_94-3+original.jpg', '0391_2-1+original.jpg', '0391_74-2+original.jpg', '0391_94-3+original.jpg', '0392_2-1+original.jpg', '0392_74-2+original.jpg', '0392_94-3+original.jpg', '0393_2-1+original.jpg', '0393_74-2+original.jpg', '0394_127-1+original.jpg', '0394_2-1+original.jpg', '0394_74-2+original.jpg', '0394_87-1+original.jpg', '0395_141-1+original.jpg', '0395_2-1+original.jpg', '0395_74-2+original.jpg', '039

##data

In [67]:
for i in tm_list:
  tum_list.append(i.split("+")[0])
del tum_list[0]

for i in nm_list:
  num_list.append(i.split("+")[0])
del num_list[0]
#print(len(tum_list),"tum_list :",tum_list) ;print(len(num_list),"num_list :",num_list);
#1069 tum_list : ['0246_141-1', '0246_87-1' // 1492 num_list : ['0602_127-1', '0602_74-2', 

In [68]:
#Masked/UnMaksed이미지에서 추출한 Cancer이미지 리스트와 Masked이미지(UnMasked)와 비교하여 UnCancer를 추출한다. 
def rm_list(Ttlist,aplist):
  tmplist=[""]
  for x in Ttlist:
    try:
      if x not in aplist:
        tmplist.append(x)
    except:
      continue
    del tmplist[0]
    print(len(tmplist),"//tmplist :",tmplist)
    return tmplist

#Modeling

##test/train/validation data 활당

In [70]:
# tr:vl:te=6:2:2  #image size (128, 128)
uncm_list=os.listdir(base_dir+"UnMasked/UnMCancer/")
unnm_list=os.listdir(base_dir+"UnMasked/UnMNormal/")
tmpnm=[""];tmpcm=[""];

#test_dir,train_dir,validation_dir
cm_list=[""];nm_list=[""];

  #이미 수행 되어 있으므로 실행말것
#cm_list=random.sample(uncm_list,int(len(uncm_list)*0.6))
#img_mv(base_dir+"UnMCancer/",train_dir+"cancer/",cm_list)
#tmpcm=rm_list(uncm_list,cm_list) ;
#nm_list=random.sample(unnm_list,int(len(unnm_list)*0.6))
#img_mv(base_dir+"UnMNormal/",train_dir+"normal/",nm_list)
#tmpnm=rm_list(uncm_list,nm_list) ;

#cm_list=random.sample(uncm_list,int(len(uncm_list)*0.2))
#img_mv(base_dir+"UnMCancer/",test_dir+"cancer/",cm_list)
#tmpcm.append(rm_list(uncm_list,cm_list)) 
#nm_list=random.sample(unnm_list,int(len(unnm_list)*0.2))
#img_mv(base_dir+"UnMNormal/",test_dir+"normal/",nm_list)
#tmpnm.append(rm_list(tmpnm,nm_list)) ;

#cm_list=random.sample(uncm_list,int(len(uncm_list)*0.2))
#img_mv(base_dir+"UnMCancer/",validation_dir+"cancer/",cm_list)
#nm_list=random.sample(unnm_list,int(len(unnm_list)*0.2))
#img_mv(base_dir+"UnMNormal/",validation_dir+"normal/",nm_list)
#print(len(uncm_list),'//',len(tmpcm),"tmpcm",tmpcm)

##Model
-저의 경우 CNN으로 작성하였지만 더 좋은 모델 찾아서 적용하는 것을 추천드립니다.
- 참고만 
    - (01/17 )기본 설정 값 test_generator ,batch_size =20 / try_t=0 ;steps_per_epoch_n=30; epochs_n=200;batch_size_n=50; validation_steps=50
tmp_ep=0.0
    - train_datagen.flow_from_directory batch_size=300, / try_t=0 ; epochs_n=300;batch_size_n=100; validation_steps=10#;steps_per_epoch_n=50
tmp_ep=0.0
- 모델 선정 또한 과제 입니다.(Unet 한번 시도는 해보는 것을 추천 드립니다. 작업하다보니 저는 시간이 안되어 시도를 못하였습니다.)
  - 아래의 모델들은 CNN으로 작성한 것으로 결과는 나올지 몰라도 모델을 사용하는 근거(목적)이 떨어집니다.

###Models1

In [72]:
from tensorflow.keras.layers import Dense

img_size=128#이는 확인된것
    #CNN modeling
model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3,3), activation='sigmoid', input_shape=(img_size, img_size, 3)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.4,seed=None),#addnoise_shape=True, 
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # #tf.keras.layers.Dense(512, activation='relu'),#,kernel_initializer='random_normal'
      #tf.keras.layers.Dense(1, activation='softmax'),
      tf.keras.layers.Dropout(0.2),#, noise_shape=None
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      ]) 

      # tf.keras.layers.MaxPooling2D(2,2),
      # tf.keras.layers.Dropout(0.6, noise_shape=None, seed=None),
      # tf.keras.layers.Flatten(),
      # #tf.keras.layers.Dense(512, activation='relu'),#,kernel_initializer='random_normal'
      # tf.keras.layers.Dense(1, activation='softmax') 
#model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=3, activation='sigmoid'))#sigmoid
model.add(tf.keras.layers.Flatten())

###Models2


In [ ]:
from tensorflow.keras.layers import Dense

img_size=300

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Dropout(0.2),#, noise_shape=None
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 149, 149, 16)     0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 35, 35, 64)      

##model compile

In [73]:
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# model compile: 
model.compile(optimizer=RMSprop(learning_rate=0.01), loss="binary_crossentropy", metrics=["accuracy"])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 30, 30, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       7

In [76]:
#img pre
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator( rescale = 1.0/200. )#horizontal_flip=True,vertical_flip=True, #상하 대칭
test_datagen  = ImageDataGenerator( rescale = 1.0/200. ,horizontal_flip=True,vertical_flip=True )
validation_datagen  = ImageDataGenerator( rescale = 1.0/200. ,horizontal_flip=True,vertical_flip=True )

#Image prep(create generation)
train_generator = train_datagen.flow_from_directory(train_dir,
            batch_size=50,  class_mode='binary',target_size=(img_size, img_size))
#test_datagen
test_generator = test_datagen.flow_from_directory(test_dir,
            batch_size=50, class_mode='binary',target_size=(img_size, img_size))

validation_generator = validation_datagen.flow_from_directory(validation_dir,
            batch_size=50, class_mode='binary',target_size=(img_size, img_size))

Found 1536 images belonging to 2 classes.
Found 511 images belonging to 2 classes.
Found 511 images belonging to 2 classes.


## working parts

In [78]:
#결과 저장하기
Train_list,Validation_list=list(),list() ; tmp_num01=0

def save_res(try_t,epochs,batch_size,steps_per_epoch,Accuracy):
#model check
  Train_list,Validation_list=list(),list()
  global tmp_num01
  loss_metrics=model.evaluate(test_generator,verbose=2)#,train_generator,
  index_n=["TRY_t","Epochs","Batch_size","Steps_per_epoch","Loss","Accuracy","File_list"]#index_n=list()
  print("index_n",len(index_n))
  
  value_d=list()
  fres=open(ori_dir+"Learnigrc.csv","a")
  #dfv=pd.DataFrame();dft=pd.DataFrame()
  for (path, dir, file) in os.walk(base_dir):
    print(path);print("dir::",dir)
    if(path[-11:]=="validation"):
          print("validation ::")
          value_d=[str(try_t)+" validation",epochs,batch_size,steps_per_epoch,loss_metrics[0],loss_metrics[1]]
          dfv=pd.DataFrame([value_d],columns=[index_n])
          print("validation //",dfv)
          if(tmp_num01==0):
            dfv.to_csv(fres,header=True,mode ="a",index=None)
            tmp_num01=1
          else: 
            dfv.to_csv(fres,header=False,mode ="a")
    elif(path[-5:]=="train"):
          print("train ::")
          value_d=[str(try_t)+" train",epochs,batch_size,steps_per_epoch,loss_metrics[0],loss_metrics[1],tm_list]
          dft=pd.DataFrame([value_d],columns=[index_n])
          print("train //",dft)
          if(tmp_num01==0):
            dft.to_csv(fres,header=False,mode ="a",index=None)
            tmp_num01=1
          else: 
            dft.to_csv(fres,header=False,mode ="a")
  # pd.merge(dft,dfv,how='outer')
  # dfv.to_csv(fres,header=False,mode ="a")
    #fres.close()

In [ ]:
#training
import matplotlib.pyplot as plt
import traceback
from tensorflow.python.keras.models import load_model

try_t=0 ; epochs_n=300;batch_size_n=50; validation_steps=2 ; steps_per_epoch_n=3
tmp_ep=0.0

res_dir=ori_dir #
while(True):
  try:
    history=model.fit(train_generator,validation_data= test_generator,
    steps_per_epoch=steps_per_epoch_n,epochs=epochs_n,verbose=2,shuffle=True )#validation_steps=20,
    
    batch_size=batch_size_n;steps_per_epoch=steps_per_epoch_n
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))
    print(epochs)
    tmp_ep=(sum(acc)/len(acc))
    
    if (try_t==0 or (sum(acc)/len(acc))>=tmp_ep):
      steps_per_epoch_n=steps_per_epoch_n+10;
      tmp_ep=(sum(acc)/len(acc))
      save_res(try_t,epochs,batch_size,steps_per_epoch,tmp_ep)
    elif((sum(acc)/len(acc))>=tmp_ep):
      batch_size_n=batch_size_n+10;
      #모델 저장하기#결과 값 지니고 있음
      model.save(res_dir+'mnist_mlp_model.h5')
      save_res(try_t,epochs,batch_size,steps_per_epoch,tmp_ep)
      tmpload_md=tf.keras.models.load_model(res_dir+'mnist_mlp_model.h5')
      test_loss, test_acc = tmpload_md.evaluate(train_generator,validation_data=validation_generator, verbose=2)#train_generator,validation_data=validation_generator, //x,  y,
      try_t=try_t+1;
    else:
      steps_per_epoch_n=steps_per_epoch_n+1;
      batch_size_n=batch_size_n+1;
      try_t=try_t+1;      
    
    plt.plot(epochs, acc, 'b', label='Training accuracy')
    plt.plot(epochs, loss, 'ro', label='Training Loss')
    plt.title('Training loss and accuracy')
    plt.legend()

    plt.figure()
    plt.plot(epochs,val_acc, 'g', label='Validation accuracy')
    plt.plot(epochs,val_loss, 'ro', label='Validation Loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()
  except Exception as em:
    print(em,"// error")
    continue                    #spe<ep&spe==vs

Epoch 1/300
3/3 - 132s - loss: 5.2059 - accuracy: 0.3400 - val_loss: 6.3563 - val_accuracy: 0.4168 - 132s/epoch - 44s/step
Epoch 2/300
3/3 - 30s - loss: 6.2014 - accuracy: 0.4067 - val_loss: 6.3563 - val_accuracy: 0.4168 - 30s/epoch - 10s/step
Epoch 3/300
3/3 - 28s - loss: 6.7097 - accuracy: 0.4400 - val_loss: 6.3563 - val_accuracy: 0.4168 - 28s/epoch - 9s/step
Epoch 4/300
3/3 - 23s - loss: 5.9980 - accuracy: 0.3933 - val_loss: 6.3563 - val_accuracy: 0.4168 - 23s/epoch - 8s/step
Epoch 5/300
